In [1]:
# import modules
import glob
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean

In [2]:
# load the json files into a dataframe
pose_keypoints = []

for filename in glob.glob('drink_jsons/*.json'):
    with open(filename, 'r') as file:
        data = json.loads(file.read())
        
    df = pd.json_normalize(data, record_path=['people'])
    pose_keypoints.append(pd.Series(df.pose_keypoints_2d[0]))
    
pose_keypoints = pd.concat(pose_keypoints, axis=1)
pose_keypoints = pose_keypoints.T

In [3]:
# drop non-essential keypoint columns
pose_keypoints = pose_keypoints.iloc[:, :76]

# add columns names
pose_keypoints.columns = [x + str(y) for y in range(25) for x in ['x', 'y', 'c']]

In [4]:
# isolate the required body parts
l_shoulder = pose_keypoints[['x5', 'y5']]
l_elbow = pose_keypoints[['x6', 'y6']]

In [5]:
# create a function that returns the indicies of the frames when movement
# started and stopped based on distance between consecutive points
def find_start_stop(xy_points):
    '''
        Takes a n x 2 Series of xy coordinates and calculates the 
        distance between consecutive points. The algorithm assumes
        that a distance >= 5 indicates actual movement. The first
        change in distance greater than 5 is marked as the start point.
        The next point where change in distance is less than 5 is
        marked as the stop point.
        
        Args:
            xy_points - n x 2 Pandas Series
            
        Returns:
            start - index of start frame
            stop - index of stop frame
    '''
    tol = 5.0
    start = -1
    stop = -1
    for i in range(len(xy_points) - 1):
        distance = (math.dist(xy_points.iloc[i], xy_points.iloc[i + 1]))
        if distance >= tol:
            start = i
            break
            
    for i in range(start, len(xy_points) - 1):
        distance = (math.dist(xy_points.iloc[i], xy_points.iloc[i + 1]))
        if distance <= tol:
            stop = i
            break
            
    return (start, stop)

In [6]:
# find the start, stop indicies
start_index, stop_index = find_start_stop(l_elbow)
print("Starting frame: ", start_index)
print("Stopping frame: ", stop_index)

Starting frame:  6
Stopping frame:  17


In [7]:
# create a function that calculates the angle between three points
def angle_calc(p0, p1, p2):
    try:
        a = (p1[0] - p0[0])**2 + (p1[1] - p0[1])**2
        b = (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2
        c = (p2[0] - p0[0])**2 + (p2[1] - p0[1])**2
        angle = math.acos((a + b - c) / math.sqrt(4 * a * b)) * 180 / math.pi
    except:
        return 0
    return round(angle, 4)

In [8]:
# calculate the angle between starting and stopping points of the left elbow
# assume left shoulder is stationary
angle = angle_calc(l_elbow.iloc[start_index], l_shoulder.mean(), l_elbow.iloc[stop_index])
print("Angle: ", angle)

Angle:  46.7153


In [9]:
# calculate movement time using the number of frames
frames = stop_index - start_index
seconds = round(frames / 30, 4)
print("Seconds: ", seconds)

Seconds:  0.3667


In [10]:
# calculate angular velocity
print("Angular velocity: ", round(angle / seconds), "degrees/second")

Angular velocity:  127 degrees/second
